In [99]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
# fetch dataset 
forest_fires = fetch_ucirepo(id=162) 
  
# data (as pandas dataframes) 
X = forest_fires.data.features 
y = forest_fires.data.targets 
  
# metadata 
print(forest_fires.metadata) 
  
# variable information 
print(forest_fires.variables) 
X.isnull().sum()

{'uci_id': 162, 'name': 'Forest Fires', 'repository_url': 'https://archive.ics.uci.edu/dataset/162/forest+fires', 'data_url': 'https://archive.ics.uci.edu/static/public/162/data.csv', 'abstract': 'This is a difficult regression task, where the aim is to predict the burned area of forest fires, in the northeast region of Portugal, by using meteorological and other data (see details at: http://www.dsi.uminho.pt/~pcortez/forestfires).', 'area': 'Climate and Environment', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 517, 'num_features': 12, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['area'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2007, 'last_updated': 'Thu Jan 11 2024', 'dataset_doi': '10.24432/C5D88D', 'creators': ['Paulo Cortez', 'Anbal Morais'], 'intro_paper': {'ID': 368, 'type': 'NATIVE', 'title': 'A data mining approach to predict forest fires using meteorological da

X        0
Y        0
month    0
day      0
FFMC     0
DMC      0
DC       0
ISI      0
temp     0
RH       0
wind     0
rain     0
dtype: int64

In [100]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['month'] = le.fit_transform(X['month'])
X['day'] = le.fit_transform(X['day'])
X

C:\Users\dbda.STUDENTSDC\AppData\Local\Temp\ipykernel_4540\1162221126.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['month'] = le.fit_transform(X['month'])
C:\Users\dbda.STUDENTSDC\AppData\Local\Temp\ipykernel_4540\1162221126.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['day'] = le.fit_transform(X['day'])


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,7,5,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,1,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0
513,2,4,1,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0
514,7,4,1,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0
515,1,4,1,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0


# Apply y as ln(1+y)

In [102]:
# y_trn_ln = np.log1p(Y_train)
# e1 = ElasticNet()
# e1.fit(X_train,y_trn_ln)
# y_pred = e1.predict(X_test)
# y_pred = np.expm1(y_pred)

In [121]:
from sklearn.experimental import enable_iterative_imputer 
#By importing enable_iterative_imputer, you explicitly acknowledge that you're using an experimental feature.
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
import numpy as np

In [123]:
it = IterativeImputer(random_state=24)
imputed_X = it.fit_transform(X)
imputed_X

array([[ 7. ,  5. ,  7. , ..., 51. ,  6.7,  0. ],
       [ 7. ,  4. , 10. , ..., 33. ,  0.9,  0. ],
       [ 7. ,  4. , 10. , ..., 33. ,  1.3,  0. ],
       ...,
       [ 7. ,  4. ,  1. , ..., 70. ,  6.7,  0. ],
       [ 1. ,  4. ,  1. , ..., 42. ,  4. ,  0. ],
       [ 6. ,  3. ,  9. , ..., 31. ,  4.5,  0. ]])

In [125]:
X_train,X_test,Y_train,Y_test = train_test_split(imputed_X,y,test_size=0.3,random_state=24)

In [127]:
y_tr = np.log1p(Y_train)

In [129]:
alpha = [0.01,0.1,0.5,1,2.5,3,5]
ratios = [0.001,0.3,0.5,0.7,0.9]
scores = [] 

for ele in alpha:
    for l1 in ratios:
        e1= ElasticNet(alpha = ele , l1_ratio= l1)
        e1.fit(X_train, y_tr)
        y_pred = e1.predict(X_test)
        y_pred = np.expm1(y_pred)
        scores.append([ele , l1 , r2_score(Y_test, y_pred)])

df = pd.DataFrame(scores, columns = ['Alpha','Ratios','r2_score'])
df.sort_values('r2_score',ascending=False).iloc[0]

Alpha       0.010000
Ratios      0.001000
r2_score   -0.023906
Name: 0, dtype: float64

In [131]:
scores

[[0.01, 0.001, -0.023905797524754968],
 [0.01, 0.3, -0.0239235762463339],
 [0.01, 0.5, -0.02393504795612511],
 [0.01, 0.7, -0.023952773533991722],
 [0.01, 0.9, -0.023974135775110206],
 [0.1, 0.001, -0.02394336078581838],
 [0.1, 0.3, -0.024233438448670874],
 [0.1, 0.5, -0.02442993638744939],
 [0.1, 0.7, -0.024598454580919027],
 [0.1, 0.9, -0.024749309855375046],
 [0.5, 0.001, -0.024116906456710607],
 [0.5, 0.3, -0.02515413246371856],
 [0.5, 0.5, -0.025630003871028206],
 [0.5, 0.7, -0.02574019884270684],
 [0.5, 0.9, -0.02581269460202873],
 [1, 0.001, -0.024289534097759802],
 [1, 0.3, -0.025704198186477623],
 [1, 0.5, -0.025849047989416674],
 [1, 0.7, -0.025950647873297372],
 [1, 0.9, -0.02597529774018792],
 [2.5, 0.001, -0.024620874458236353],
 [2.5, 0.3, -0.025957220972347983],
 [2.5, 0.5, -0.026018241493104277],
 [2.5, 0.7, -0.026078027027035766],
 [2.5, 0.9, -0.026136555836524966],
 [3, 0.001, -0.024695526550924995],
 [3, 0.3, -0.02597577965682696],
 [3, 0.5, -0.026048382967899908],
 